# MiniDataAPI Spec

The `MiniDataAPI` is a persistence API specification that designed to be small and relatively easy to implement across a wide range of datastores. This means that while early implementations have been SQL-based, the specification can be quickly implemented in key/value stores, document databases, and more.



In [40]:
#| hide
from fasthtml.common import *

## Connect/construct the database

We connect or construct the database by passing in a string connecting to the database endpoint or a filepath representing the database's location. While this example is for a Sqlite database on a harddrive, other databases such as PostgreSQL, Redis, MongoDB, would instead use a URI pointing at the database's endpoint.

In [41]:
db = database('data/minidataapi.db')

## Create database tables

Here we use Python classes to define representations of objects. While in this example we use strings and integers, the MiniDataAPI specification doesn't include types.

In [42]:
class User: name:str; email: str; year_started:int
users = db.create(User, pk='name')

class Todo: id: int; title: str; detail: str; status: str; name: str
# If no `pk` is provided, id is assumed to be the primary key
todos = db.create(Todo) 

In [43]:
#| hide
# Remove records
[users.delete(o['name']) for o in users.rows]
[todos.delete(o['id']) for o in todos.rows]

[<Table todo (id, title, detail, status, name)>,
 <Table todo (id, title, detail, status, name)>,
 <Table todo (id, title, detail, status, name)>]

## insert

Add a new record to the database. Must accept Python dictionaries, dataclasses, and standard classes. Returns an instance of the new record.

In [44]:
# Add records to the users table
users.insert({'name': 'Braden', 'email': 'b@example.com', 'year_started': 2018})
users.insert({'name': 'Alma', 'email': 'a@example.com', 'year_started': 2019})
users.insert({'name': 'Charlie', 'email': 'c@example.com', 'year_started': 2018})


User(name='Charlie', email='c@example.com', year_started=2018)

In [45]:
# Now the todos table
todos.insert(dict(title='Write MiniDataAPI spec', status='open', name='Braden'))
todos.insert(dict(title='Implement SSE in FastHTML', status='open', name='Alma'))
todos.insert(dict(title='Launch FastHTML', status='closed', name='Charlie'))


Todo(id=3, title='Launch FastHTML', detail=None, status='closed', name='Charlie')

## Square bracket search

Get a single record by entering a primary key into a table object within square brackets. 

In [46]:
users['Alma']

User(name='Alma', email='a@example.com', year_started=2019)

If no record is found, a `NotFoundError` error is raised. 

In [47]:
name = 'David'
try:
    users[name]
except NotFoundError:
    print(f'user {name} not found')

user David not found


And now a demonstration of a ticket search, demonstrating how this works with non-string primary keys. 

In [48]:
todos[1]

Todo(id=1, title='Write MiniDataAPI spec', detail=None, status='open', name='Braden')

## Circle bracket search

Get zero to many records by entering values with circle bracket searches.

In [49]:
for user in users():
    print(user)

User(name='Braden', email='b@example.com', year_started=2018)
User(name='Alma', email='a@example.com', year_started=2019)
User(name='Charlie', email='c@example.com', year_started=2018)


We can order the results.

In [50]:
for user in users(order_by='name'):
    print(user)

User(name='Alma', email='a@example.com', year_started=2019)
User(name='Braden', email='b@example.com', year_started=2018)
User(name='Charlie', email='c@example.com', year_started=2018)


We can filter on the results:

In [51]:
for user in users(where="year_started=2019"):
    print(user)

User(name='Alma', email='a@example.com', year_started=2019)


## update

Update an existing record of the database. Must accept Python dictionaries, dataclasses, and standard classes. Uses the primary key for identifying the record to be changed. Returns an instance of the updated record. 

In [52]:
users.update(User(name='Alma', year_started=1899))


User(name='Alma', email=None, year_started=1899)

If the primary key doesn't match a record, raise a `NoteFoundError`.

In [53]:
try:
    users.update(User(name='John', year_started=2024))
except NotFoundError:
    print('User not found')

User not found


## delete

Delete a record of the database. Uses the primary key for identifying the record to be removed. Does not return anything.

In [54]:
try:
    users.delete('John')
except NotFoundError:
    print('User not deleted')

User not deleted


If the primary key value can't be found, raises a `NotFoundError`.

In [55]:
users.delete('Alma')

<Table user (name, email, year_started)>

# xtra

The xtra action adds a filter to queries and DDL statements. This makes it easier to limit users (or other objects) access to only things for which they have permission.

## Implementations

- [fastlite](https://github.com/AnswerDotAI/fastlite) - The original implementation, only for Sqlite
- [fastsql](https://github.com/AnswerDotAI/fastsql) - An SQL database agnostic implementation based on the excellent SQLAlchemy library.